In [1]:
import requests
import cnfg
import json
import pickle
import time
from datetime import datetime
from collections import defaultdict
from urlparse import urlparse
from pymongo import MongoClient

In [2]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [3]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [4]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [5]:
rap_caviar = [2400811, 2415289, 2342860, 2369079, 2411389, 2304247, 2396302, 599451, \
              2398213, 2433137, 2330692, 2392485, 2348651, 2295548, 2372618, 2425602, \
              2439141, 2165110, 2375319, 2312297, 2339761, 2394578, 2305085, 2422646, \
              2347642, 2396122, 2307164, 332635, 2347598, 1783676, 2399225, 2386805, 639247, \
              2432858, 709256, 2403555, 2163216, 2417517, 2401700, 2213105, 2312325, 2414510, \
              2426159, 2426609, 2424059, 2344225, 2440578, 2433471, 2421937, 2413367]

In [7]:
def get_annotations(song):
    referenced = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/referents?song_id='+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for ref in refs_json['response']['referents']:
        for note in ref['annotations']:
            for child in note['body']['dom']['children']:
                if 'children' in child:
                    for element in child['children']:
                        if 'attributes' in element:
                            if 'href' in element['attributes']:
                                parsed = urlparse(element['attributes']['href'])
                                if parsed.netloc=="genius.com":
                                    apiurl = req_start+element['data']['api_path']
                                    annotype = urlparse(apiurl).path.split('/')[1]
                                    if annotype == 'albums':
                                        referenced['albums'].append(apiurl.encode('utf-8'))
                                    elif annotype == 'artists':
                                        referenced['artists'].append(apiurl.encode('utf-8'))
                                    elif annotype == 'songs':
                                        if urlparse(element['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                            referenced['songs'].append(apiurl.encode('utf-8'))
                                        else:
                                            referenced['other'].append(apiurl.encode('utf-8'))
                                            
    return dict(referenced)

In [8]:
def description_refs(song):
    description = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/songs/'+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for paragraph in refs_json['response']['song']['description']['dom']['children']:
        if 'children' in paragraph:
            for child in paragraph['children']:
                if 'attributes' in child:
                    if 'href' in child['attributes']:
                        url = child['attributes']['href']
                        if urlparse(url).netloc=='genius.com':
                            apiurl = req_start+child['data']['api_path']
                            annotype = urlparse(apiurl).path.split('/')[1]
                            if annotype == 'albums':
                                description['albums'].append(apiurl.encode('utf-8'))
                            elif annotype == 'artists':
                                description['artists'].append(apiurl.encode('utf-8'))
                            elif annotype == 'songs':
                                if urlparse(child['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                    description['songs'].append(apiurl.encode('utf-8'))
                                else:
                                    description['other'].append(apiurl.encode('utf-8'))
    return dict(description)

In [11]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

In [9]:
# pass one of the two following:
# song['references'] for referents or
# song['description_refs'] for references in description

def get_neighbors(song):
    neighbors = defaultdict(list)
    for item, vals in song.iteritems():
        if item=='artists':
            for artist in vals:
                artistid = artist.rpartition('/')[2]
                if s.find({'featured_artists.id': artistid}, {'featured_artists.artist.$': 1}).count():
                    name = s.find({'featured_artists.id': artistid}, {'featured_artists.artist.$': 1}).find()
                    print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
                elif s.find({'writers.id': artistid}, {'writers.artist.$': 1}).count():
                    name = s.find({'writers.id': artistid}, {'writers.artist.$': 1}).find()
                    print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
                elif s.find({'producers.id': artistid}, {'producers.artist.$': 1}).count():
                    name = s.find({'producers.id': artistid}, {'producers.artist.$': 1}).find()
                    print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
                elif s.find({'artist_id': artistid}, {'artist.$':1}).count():
                    name = s.find({'artist_id': artistid}, {'artist.$': 1}).find()
                    print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
                else:
                    print "calling artistid: ", artistid
                    time.sleep(7)
                    name = api_call(artist)['response']['artist']['name']
                    print "successful request!"
                    print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
        elif item == 'albums':
            for album in vals:
                albumid = album.rpartition('/')[2]
                if s.find({'album.id': albumid}, {'album.name.$': 1}).count():
                    name = s.find({'album.id': albumid}, {'album.name.$': 1})
                    print "album name: ", name
                    neighbors['album'].append({'name': name, 'id': albumid})
                else:
                    print "calling albumid: ", albumid
                    time.sleep(7)
                    name = api_call(album)['response']['album']['name']
                    print "successful request!"
                    print "album name: ", name
                    neighbors['album'].append({'name': name, 'id': albumid})
        elif item == 'songs':
            for song in vals:
                songid = song.rpartition('/')[2]
                if s.find({'id': songid}).count(): 
                    name = s.find({'id': songid}, {'name.$':1}).next()
                    artist = s.find({'id':songid}, {'artist.$':1}).next()
                    print "songs already exists: ", songid, " title: ", name
                    neighbors['song'].append({'title':name, 'artist':artist, 'id':songid})
                else:
                    neighbors['song_unannotated'].append({'id':songid})
#                     print "annotating song: ", songid
#                     time.sleep(7)
#                     s.insert_one(annotate_song(songid))
#                     name = s.find({'id': songid}, {'title.$':1}).next()
#                     artist = s.find({'id':songid}, {'artist.$':1}).next()
#                     neighbors['song'].append({'title':name, 'artist':artist, 'id':songid})
        elif item == 'other':
            for other in vals:
                otherid = other.rpartition('/')[2]
                print "calling other: ", other
                name = api_call(other)['response']['song']['title']
                neighbors['other'].append({'title':name, 'id':otherid})
                
    return neighbors

In [39]:
def annotate_song(songid):
    if s.find({'id':songid}).count() > 0:
        print "already in Mongo... returning Mongo query"
        return s.find({'id':songid}).next()
    else:
        song_info = {}
        myrequest = req_start+'/songs/'+str(songid)
        r = requests.get(myrequest, headers={'Authorization':auth})
        song_json = json.loads(r.content)
        song_info['title'] = str(song_json['response']['song']['title'])
        song_info['artist'] = str(song_json['response']['song']['primary_artist']['name'])
        song_info['artist_id'] = song_json['response']['song']['primary_artist']['id']
        pickle_title = '_'.join(song_info['title'].split())+"_"+'_'.join(song_info['artist'].split())+".pkl"
        pickleDump(song_json, pickle_title)
        if song_json['response']['song']['album'] != None:
            song_info['album'] = {'name': str(song_json['response']['song']['album']['name']), 
                              'id': song_json['response']['song']['album']['id']}
        song_info['id'] = song_json['response']['song']['id']

        # may or may not exist
        song_info['featured_artists'] = []
        song_info['writers'] = []
        song_info['producers'] = []
        
        if song_json['response']['song']['sampled_songs']:
            song_info['sampled_songs'] = song_json['response']['song']['sampled_songs']
        if song_json['response']['song']['sampling_songs']:
            song_info['sampling_songs'] = song_json['response']['song']['sampling_songs']

        for artist in song_json['response']['song']['featured_artists']:
            if artist['name']!=song_info['artist']:
                song_info['featured_artists'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['producer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['producers'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['writer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['writers'].append({'artist':artist['name'], 'id':artist['id']})

        if not song_info['featured_artists']:
            del song_info['featured_artists']
        if not song_info['writers']:
            del song_info['writers']
        if not song_info['producers']:
            del song_info['producers']

        # references!!
        time.sleep(7)
        song_info['references'] = dict(get_neighbors(get_annotations(songid)))
        time.sleep(7)
        song_info['description_refs'] = dict(get_neighbors(description_refs(song_json)))
        
        pickled_annotation = pickle_title[0:-4]+"_annotated.pkl"
        pickleDump(song_info, pickled_annotation)
        return song_info

In [13]:
s.find({'id':2396122}).count()

1

In [18]:
for songid in rap_caviar:
    if s.find({'id':songid}).count()>0:
        print "song id ", songid, " in Mongo already"
        continue
    else:
        try:
            annotated = annotate_song(songid)
            s.insert_one(annotated)
            print "inserted ", songid, " to Mongo!"
            time.sleep(7)
        except:
            print "unable to insert: ", songid
            continue

unable to insert:  2400811
calling artistid:  1632
successful request!
artist name:  Tory Lanez
calling artistid:  1632
successful request!
artist name:  Tory Lanez
calling artistid:  27638
successful request!
artist name:  KeY Wane
calling artistid:  1319
successful request!
artist name:  Meek Mill
calling albumid:  120604
successful request!
album name:  The Life of Pablo
calling other:  https://api.genius.com/songs/1791
calling albumid:  132506
successful request!
album name:  Views From The 6
calling other:  https://api.genius.com/songs/2415314
inserted  2415289  to Mongo!
unable to insert:  2342860
calling albumid:  122065
successful request!
album name:  Barter 6
calling artistid:  44080
successful request!
artist name:  Migos
inserted  2369079  to Mongo!
inserted  2411389  to Mongo!
calling artistid:  11395
successful request!
artist name:  G-Eazy
calling artistid:  1073
successful request!
artist name:  Juicy J
calling artistid:  27486
successful request!
artist name:  TM88
cal

In [19]:
unable_to_insert = [2400811, 2342860, 2439141, 2339761, 2347642, 332635, 2347598, 2399225, \
                    2386805, 639247, 2432858, 2403555, 2163216, 2213105, 2312325, 2414510, 2426159, 2421937]

In [20]:
50-len(unable_to_insert)+1

33

In [21]:
s.count()

33

In [22]:
if 2347598 in unable_to_insert:
    print True

True


In [24]:
blt = 2347598

In [27]:
urlblt = req_start+'/songs/'+str(blt)

In [28]:
blt_call = api_call(urlblt)

In [36]:
blt_call['response']['song']['album']

In [38]:
if blt_call['response']['song']['sampled_songs']:
    print True
else:
    print False

False
